In [1]:
#Imports
import pandas as pd
import requests
import matplotlib.pyplot as plt
import json
import numpy as np
from pandas.api.types import is_string_dtype
import os
import seaborn as sns
from sklearn.ensemble import GradientBoostingClassifier
import numpy as np
import pandas as pd
%matplotlib inline
plt.rcParams['font.size'] = 24
from IPython.core.pylabtools import figsize
from sklearn.preprocessing import StandardScaler,Normalizer,FunctionTransformer,QuantileTransformer,PowerTransformer
from sklearn import model_selection
from sklearn.model_selection import RepeatedKFold
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split,cross_validate
from sklearn.metrics import accuracy_score, confusion_matrix, r2_score
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import Pipeline
import warnings
from sklearn.svm import SVR
import statsmodels.api as sm

In [2]:
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
dfc=pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/All_data_2014-2019.csv')
iden=pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Energy_data_2020.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (10,195,200,229,230,231) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
red=iden[['property_id','largest_property_use_type_gross_floor_area_ft']]
#red['property_id']=red['property_id'].astype(int)
#red['property_id'] = red['property_id'].astype(str)


In [5]:
#dfc.drop(['Unnamed: 0'],axis=1,inplace=True)
dfc.head()

,Unnamed: 0,property_id,property_name,year_ending,nyc_building_identification,nyc_borough_block_and_lot,address_1,city,postcode,primary_property_type_self,...,net_emissions_metric_tons,green_power_offsite_kwh,borough,latitude,longitude,community_board,council_district,census_tract,nta,Year
0,0,7365,1155,2014-12-31T00:00:00.000,1022631,1009970029,1155 Avenue of the Americas,Manhattan,10036,Office,...,3054.3,0.0,MANHATTAN,40.756631,-73.982826,105.0,4.0,119.0,Midtown-Midtown South,2014
1,1,8139,200,2014-12-31T00:00:00.000,1037545,1013150001,200 East 42nd St.,Manhattan,10017,Office,...,2453.3,0.0,MANHATTAN,40.750698,-73.974306,106.0,4.0,88.0,Turtle Bay-East Midtown,2014
2,2,8604,114,2014-12-31T00:00:00.000,1022667,1009990019,114 West 47th st,Manhattan,10036,Office,...,2961.2,0.0,MANHATTAN,40.758310,-73.982504,105.0,4.0,125.0,Midtown-Midtown South,2014
3,3,8841,733,2014-12-31T00:00:00.000,1037596,1013190047,733 Third Avenue,Manhattan,10017,Office,...,2731.4,0.0,MANHATTAN,40.753074,-73.972753,106.0,4.0,90.0,Turtle Bay-East Midtown,2014
4,4,11809,Conde Nast Building,2014-12-31T00:00:00.000,1085682,1009950005,4 Times Square,Manhattan,10036,Office,...,13612.9,132841.1,MANHATTAN,40.756181,-73.986244,105.0,4.0,119.0,Midtown-Midtown South,2014


In [6]:
total = pd.merge(dfc, red, on=["property_id"])

In [7]:
total.columns

Index(['Unnamed: 0', 'property_id', 'property_name', 'year_ending',
       'nyc_building_identification', 'nyc_borough_block_and_lot', 'address_1',
       'city', 'postcode', 'primary_property_type_self', 'year_built',
       'number_of_buildings', 'occupancy', 'metered_areas_energy',
       'energy_star_score', 'site_eui_kbtu_ft', 'weather_normalized_site_eui',
       'site_energy_use_kbtu', 'weather_normalized_site_energy',
       'weather_normalized_site', 'source_eui_kbtu_ft',
       'weather_normalized_source', 'source_energy_use_kbtu',
       'natural_gas_use_kbtu', 'electricity_use_grid_purchase',
       'avoided_emissions_offsite', 'estimated_data_flag',
       'estimated_data_flag_natural', 'total_ghg_emissions_metric',
       'direct_ghg_emissions_metric', 'indirect_ghg_emissions_metric',
       'egrid_output_emissions_rate', 'net_emissions_metric_tons',
       'green_power_offsite_kwh', 'borough', 'latitude', 'longitude',
       'community_board', 'council_district', 'census

In [8]:
next=total[['Year','property_id','borough','primary_property_type_self','year_built','occupancy','energy_star_score','natural_gas_use_kbtu','electricity_use_grid_purchase','egrid_output_emissions_rate','green_power_offsite_kwh','largest_property_use_type_gross_floor_area_ft','latitude', 'longitude']]

In [9]:
dfc.columns

Index(['Unnamed: 0', 'property_id', 'property_name', 'year_ending',
       'nyc_building_identification', 'nyc_borough_block_and_lot', 'address_1',
       'city', 'postcode', 'primary_property_type_self', 'year_built',
       'number_of_buildings', 'occupancy', 'metered_areas_energy',
       'energy_star_score', 'site_eui_kbtu_ft', 'weather_normalized_site_eui',
       'site_energy_use_kbtu', 'weather_normalized_site_energy',
       'weather_normalized_site', 'source_eui_kbtu_ft',
       'weather_normalized_source', 'source_energy_use_kbtu',
       'natural_gas_use_kbtu', 'electricity_use_grid_purchase',
       'avoided_emissions_offsite', 'estimated_data_flag',
       'estimated_data_flag_natural', 'total_ghg_emissions_metric',
       'direct_ghg_emissions_metric', 'indirect_ghg_emissions_metric',
       'egrid_output_emissions_rate', 'net_emissions_metric_tons',
       'green_power_offsite_kwh', 'borough', 'latitude', 'longitude',
       'community_board', 'council_district', 'census

In [10]:
um=next.groupby(['property_id'])['energy_star_score'].count().reset_index(name='count')

In [12]:
id=[]
for i in range(len(um['property_id'])):
  if um['count'].iloc[i]>=5:
    id.append(um['property_id'].iloc[i])

In [13]:
len(id)

8554

In [ ]:
#view all rows and columns
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', 199)  # or 199

In [115]:
dfc1=dfc.drop_duplicates()

In [116]:
check=dfc1[dfc1['property_id'].isin(id)]
check.groupby(['borough','primary_property_type_self']).count()

Unnamed: 0  property_id  property_name  \
borough   primary_property_type_self                                           
BRONX     College/University                1064         1064           1064   
          Distribution Center                104          104            104   
          Food Service                         4            4              4   
          Hotel                                2            2              2   
          Multifamily Housing               8228         8228           8228   
          Office                             199          199            199   
          Other                                8            8              8   
          Recreation                           2            2              2   
          Residence Hall/Dormitory            59           59             59   
          Retail Store                        63           63             63   
          Senior Care Community              159          159            159   
BROOKLYN  College/University                2130         2130           2130   
          Distribution Center                321          321            321   
          Food Service                        24           24             24   
          Hotel                               29           29             29   
          Multifamily Housing               8617         8617           8617   
          Office                             269          269            269   
          Other                               29           29             29   
          Recreation                          39           39             39   
          Residence Hall/Dormitory           173          173            173   
          Retail Store                        62           62             62   
          Senior Care Community              165          165            165   
MANHATTAN College/University                1016         1016           1016   
          Distribution Center                 38           38             38   
          Food Service                        10           10             10   
          Hotel                              770          770            770   
          Multifamily Housing              13078        13078          13078   
          Office                            4918         4918           4918   
          Other                               49           49             49   
          Recreation                          20           20             20   
          Residence Hall/Dormitory           494          494            494   
          Retail Store                        64           64             64   
          Senior Care Community               69           69             69   
QUEENS    College/University                1273         1273           1273   
          Distribution Center                405          405            405   
          Food Service                        25           25             25   
          Hotel                               57           57             57   
          Multifamily Housing               6079         6079           6079   
          Office                             313          313            313   
          Other                               58           58             58   
          Recreation                          12           12             12   
          Residence Hall/Dormitory            18           18             18   
          Retail Store                        92           92             92   
          Senior Care Community              187          187            187   
STATEN IS College/University                 291          291            291   
          Distribution Center                 10           10             10   
          Food Service                        27           27             27   
          Hotel                               11           11             11   
          Multifamily Housing         

In [117]:
check[check['borough']=='STATEN IS'].groupby(['Year','primary_property_type_self']).count()

Unnamed: 0  property_id  property_name  \
Year primary_property_type_self                                           
2014 College/University                   2            2              2   
     Food Service                         2            2              2   
     Hotel                                1            1              1   
     Multifamily Housing                 37           37             37   
     Office                               8            8              8   
     Other                                4            4              4   
     Retail Store                         4            4              4   
     Senior Care Community                6            6              6   
2015 College/University                  57           57             57   
     Distribution Center                  1            1              1   
     Food Service                         3            3              3   
     Hotel                                2            2              2   
     Multifamily Housing                 47           47             47   
     Office                               9            9              9   
     Other                                1            1              1   
     Retail Store                         6            6              6   
     Senior Care Community               10           10             10   
2016 College/University                  57           57             57   
     Distribution Center                  2            2              2   
     Food Service                         3            3              3   
     Hotel                                2            2              2   
     Multifamily Housing                 48           48             48   
     Office                              10           10             10   
     Other                                1            1              1   
     Residence Hall/Dormitory             1            1              1   
     Retail Store                         5            5              5   
     Senior Care Community                8            8              8   
2017 College/University                  61           61             61   
     Distribution Center                  5            5              5   
     Food Service                         7            7              7   
     Hotel                                2            2              2   
     Multifamily Housing                 93           93             93   
     Office                              14           14             14   
     Other                                5            5              5   
     Residence Hall/Dormitory             2            2              2   
     Retail Store                         6            6              6   
     Senior Care Community               19           19             19   
2018 College/University                  57           57             57   
     Distribution Center                  1            1              1   
     Food Service                         5            5              5   
     Hotel                                2            2              2   
     Multifamily Housing                 52           52             52   
     Office                              10           10             10   
     Other                                1            1              1   
     Residence Hall/Dormitory             1            1              1   
     Retail Store                         6            6              6   
     Senior Care Community                8            8              8   
2019 College/University                  57           57             57   
     Distribution Center                  1            1              1   
     Food Service                         7            7              7   
     Hotel                                2            2              2   
     Multifamily Housing                 55           55  

In [17]:
property_type = {'Multifamily Housing':'Multifamily Housing',  
            'Residence Hall/Dormitory':'College/University',
            'Other - Lodging/Residential':'College/University',
            'Hotel':'Hotel',
            'Adult Education':'College/University',
            'College/University':'College/University',
            'K-12 School':'College/University',
            'Library':'College/University',
            'Vocational School':'College/University',
            'Other - Education':'College/University',
            'Office':'Office',
            'Medical Office':'Office',
            'Financial Office':'Office',
            'Bank Branch':'Office',
            'Distribution Center':'Distribution Center',
            'Self-Storage Facility':'Distribution Center',
            'Wholesale Club/Supercenter':'Distribution Center',
            'Non-Refrigerated Warehouse':'Distribution Center',
            'Fast Food Restaurant':'Food Service',
            'Food Sales':'Food Service',
            'Food Service':'Food Service',
            'Restaurant':'Food Service',
                 'Supermarket/Grocery Store':'Food Service',
            'Convenience Store without Gas Station':'Food Service',
            'Other - Restaurant/Bar':'Food Service',
            'Hospital (General Medical & Surgical)':'Senior Care Community',
            'Urgent Care/Clinic/Other Outpatient':'Senior Care Community',
            'Ambulatory Surgical Center':'Senior Care Community',
            'Laboratory':'Senior Care Community',
            'Pre-school/Daycare':'Senior Care Community',
            'Senior Care Community':'Senior Care Community',
            'Outpatient Rehabilitation/Physical Therapy':'Senior Care Community',
            'Retail Store':'Retail Store',
            'Repair Services (Vehicle, Shoe, Locksmith, etc.)':'Retail Store',
            'Mailing Center/Post Office':'Retail Store',
            'Automobile Dealership':'Retail Store',
            'Mailing Center/Post Office':'Retail Store',
            'Personal Services (Health/Beauty, Dry Cleaning...':'Retail Store',
            'Enclosed Mall':'Retail Store',
            'Other - Mall':'Retail Store',
            'Other - Services':'Retail Store',
            'Other - Utility':'Retail Store',
            'Bar/Nightclub':'Recreation',
            'Bowling Alley':'Recreation',
                 'Fitness Center/Health Club/Gym':'Recreation',
            'Other - Recreation':'Recreation',
            'Other - Entertainment/Public Assembly':'Recreation',
            'Performing Arts':'Recreation',
            'Social/Meeting Hall':'Recreation',
            'Museum':'Recreation',
            'Worship Facility':'Recreation',
            'Other':'Other',
            'Courthouse':'Other',
            'Other - Public Services':'Other',
            'Swimming Pool':'Other',
            'Parking':'Other',
            'Refrigerated Warehouse':'Other',
            'Data Center':'Other',
            'none':'none'
              }

In [16]:
check['primary_property_type_self'].unique()

array(['Office', 'Senior Care Community', 'Retail Store',
       'Multifamily Housing', 'Hotel', 'College/University',
       'Residence Hall/Dormitory', 'Distribution Center', 'Other', nan,
       'Food Service', 'Recreation'], dtype=object)

In [18]:
check['primary_property_type_self'] = check['primary_property_type_self'].map(property_type).astype(str)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [19]:
check['primary_property_type_self'].unique()

array(['Office', 'Senior Care Community', 'Retail Store',
       'Multifamily Housing', 'Hotel', 'College/University',
       'Distribution Center', 'Other', 'nan', 'Food Service'],
      dtype=object)

In [ ]:
check.groupby(['borough','primary_property_type_self']).count()

In [21]:
import random


In [ ]:
#um=next.groupby(['property_id'])['energy_star_score'].count().reset_index(name='count')

In [ ]:
check['borough'].iloc[1]

'MANHATTAN'

In [22]:
#multifam by borough 50 each
bk_mf_id=[]
for i in range(len(check)): 
   if check['borough'].iloc[i]=='BROOKLYN' and check['primary_property_type_self'].iloc[i]=='Multifamily Housing':
     bk_mf_id.append(check['property_id'].iloc[i])
bx_mf_id=[]
for i in range(len(check)): 
   if check['borough'].iloc[i]=='BRONX' and check['primary_property_type_self'].iloc[i]=='Multifamily Housing':
     bx_mf_id.append(check['property_id'].iloc[i])
mn_mf_id=[]
for i in range(len(check)): 
   if check['borough'].iloc[i]=='MANHATTAN' and check['primary_property_type_self'].iloc[i]=='Multifamily Housing':
     mn_mf_id.append(check['property_id'].iloc[i])
si_mf_id=[]
for i in range(len(check)): 
   if check['borough'].iloc[i]=='STATEN IS' and check['primary_property_type_self'].iloc[i]=='Multifamily Housing':
     si_mf_id.append(check['property_id'].iloc[i])
qn_mf_id=[]
for i in range(len(check)): 
   if check['borough'].iloc[i]=='QUEENS' and check['primary_property_type_self'].iloc[i]=='Multifamily Housing':
     qn_mf_id.append(check['property_id'].iloc[i])


In [23]:
tot_mf=[]
tot_mf.append(random.sample(set(bk_mf_id), 50))
tot_mf.append(random.sample(set(bx_mf_id), 50))
tot_mf.append(random.sample(set(mn_mf_id), 50))
tot_mf.append(random.sample(set(si_mf_id), 50))
tot_mf.append(random.sample(set(qn_mf_id), 50))

In [24]:
mf_ids=sum(tot_mf,[])
len(mf_ids)

250

In [25]:
#office by borough 30 each
bk_off_id=[]
for i in range(len(check)): 
   if check['borough'].iloc[i]=='BROOKLYN' and check['primary_property_type_self'].iloc[i]=='Office':
     bk_off_id.append(check['property_id'].iloc[i])
bx_off_id=[]
for i in range(len(check)): 
   if check['borough'].iloc[i]=='BRONX' and check['primary_property_type_self'].iloc[i]=='Office':
     bx_off_id.append(check['property_id'].iloc[i])
mn_off_id=[]
for i in range(len(check)): 
   if check['borough'].iloc[i]=='MANHATTAN' and check['primary_property_type_self'].iloc[i]=='Office':
     mn_off_id.append(check['property_id'].iloc[i])
si_off_id=[]
for i in range(len(check)): 
   if check['borough'].iloc[i]=='STATEN IS' and check['primary_property_type_self'].iloc[i]=='Office':
     si_off_id.append(check['property_id'].iloc[i])
qn_off_id=[]
for i in range(len(check)): 
   if check['borough'].iloc[i]=='QUEENS' and check['primary_property_type_self'].iloc[i]=='Office':
     qn_off_id.append(check['property_id'].iloc[i])

In [28]:
len(set(si_off_id))

10

In [33]:
tot_off=[]
tot_off.append(random.sample(set(bk_off_id), 30))
tot_off.append(random.sample(set(bx_off_id), 30))
tot_off.append(random.sample(set(mn_off_id), 30))
tot_off.append(random.sample(set(si_off_id), 10))
tot_off.append(random.sample(set(qn_off_id), 30))

In [ ]:
set(si_off_id)

{2797222,
 2798104,
 2926168,
 3098815,
 3121506,
 3121988,
 3237564,
 3237565,
 4389084,
 4880646}

In [34]:
next[next['property_id']==4389084]

,Year,property_id,borough,primary_property_type_self,year_built,occupancy,energy_star_score,natural_gas_use_kbtu,electricity_use_grid_purchase,egrid_output_emissions_rate,green_power_offsite_kwh,largest_property_use_type_gross_floor_area_ft,latitude,longitude
43251,2014,4389084,STATEN IS,Office,1974,100,75.0,3325459.7,1768428.6,82.9,0.0,79150.0,40.643264,-74.077159
43252,2015,4389084,STATEN IS,Office,1974,100,78.0,2010249.9,2122098.6,82.9,0.0,79150.0,40.643264,-74.077159
43253,2016,4389084,STATEN IS,Office,1974,100,77.0,1607330.6,2237941.2,92.8,0.0,79150.0,40.643264,-74.077159
43254,2017,4389084,STATEN IS,Office,1974,100,76.0,1003491.0,2435931.4,88.7,0.0,79150.0,40.643264,-74.077159
43255,2018,4389084,STATEN IS,Office,1974,100,58.0,245880.0,2695841.5,84.7,0.0,79150.0,40.643264,-74.077159
43256,2019,4389084,STATEN IS,Office,1974,100,66.0,1543499.8,2428726.9,84.7,0.0,79150.0,40.643264,-74.077159


In [35]:
off_ids=sum(tot_off,[])
len(off_ids)

130

In [36]:
#College/Dorm by borough 50 each
bk_col_id=[]
for i in range(len(check)): 
   if check['borough'].iloc[i]=='BROOKLYN' and check['primary_property_type_self'].iloc[i]=='College/University':
     bk_col_id.append(check['property_id'].iloc[i])
bx_col_id=[]
for i in range(len(check)): 
   if check['borough'].iloc[i]=='BRONX' and check['primary_property_type_self'].iloc[i]=='College/University':
     bx_col_id.append(check['property_id'].iloc[i])
mn_col_id=[]
for i in range(len(check)): 
   if check['borough'].iloc[i]=='MANHATTAN' and check['primary_property_type_self'].iloc[i]=='College/University':
     mn_col_id.append(check['property_id'].iloc[i])
si_col_id=[]
for i in range(len(check)): 
   if check['borough'].iloc[i]=='STATEN IS' and check['primary_property_type_self'].iloc[i]=='College/University':
     si_col_id.append(check['property_id'].iloc[i])
qn_col_id=[]
for i in range(len(check)): 
   if check['borough'].iloc[i]=='QUEENS' and check['primary_property_type_self'].iloc[i]=='College/University':
     qn_col_id.append(check['property_id'].iloc[i])

In [37]:
tot_col=[]
tot_col.append(random.sample(set(bk_col_id), 50))
tot_col.append(random.sample(set(bx_col_id), 50))
tot_col.append(random.sample(set(mn_col_id), 50))
tot_col.append(random.sample(set(si_col_id), 50))
tot_col.append(random.sample(set(qn_col_id), 50))

In [38]:
col_ids=sum(tot_col,[])
len(col_ids)

250

In [39]:
final_ids=col_ids+mf_ids+off_ids


In [40]:
final_data=next[next['property_id'].isin(final_ids)]
final_data.head()

,Year,property_id,borough,primary_property_type_self,year_built,occupancy,energy_star_score,natural_gas_use_kbtu,electricity_use_grid_purchase,egrid_output_emissions_rate,green_power_offsite_kwh,largest_property_use_type_gross_floor_area_ft,latitude,longitude
263,2014,1407848,MANHATTAN,Office,1963,100,53.0,8100654.8,211648520.8,82.9,0.0,1733828.0,40.754162,-73.976557
264,2015,1407848,MANHATTAN,Office,1963,85,55.0,7430345.4,173812489.7,82.9,0.0,1733828.0,40.754162,-73.976557
265,2016,1407848,MANHATTAN,Office,1963,85,72.0,7224061.1,137394745.4,92.8,0.0,1733828.0,40.754162,-73.976557
266,2017,1407848,MANHATTAN,Office,1963,85,59.0,7225638.6,159928080.3,88.7,0.0,1733828.0,40.754162,-73.976557
267,2018,1407848,MANHATTAN,Office,1963,85,32.0,7244369.0,161546077.8,84.7,0.0,1733828.0,40.754162,-73.976557


In [47]:
final_data.reset_index(drop=True, inplace=True)

In [48]:
final_data[(final_data['borough']=='STATEN IS') & (final_data['primary_property_type_self']=='Office')]

,Year,property_id,borough,primary_property_type_self,year_built,occupancy,energy_star_score,natural_gas_use_kbtu,electricity_use_grid_purchase,egrid_output_emissions_rate,green_power_offsite_kwh,largest_property_use_type_gross_floor_area_ft,latitude,longitude
897,2014,2797222,STATEN IS,Office,1925,100,48.0,504558.1,4913903.7,82.9,0.0,61291.0,40.644518,-74.110781
898,2015,2797222,STATEN IS,Office,1925,100,47.0,487603.2,5029017.3,82.9,0.0,61291.0,40.644518,-74.110781
899,2016,2797222,STATEN IS,Office,1925,100,50.0,283100.0,4849829.6,92.8,0.0,61291.0,40.644518,-74.110781
900,2017,2797222,STATEN IS,Office,1925,100,29.0,3395499.9,5027663.6,88.7,0.0,61291.0,40.644518,-74.110781
901,2018,2797222,STATEN IS,Office,1925,100,32.0,273800.0,5015877.1,84.7,0.0,61291.0,40.644518,-74.110781
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2404,2015,4880646,STATEN IS,Office,1975,100,29.0,10433999.9,5396443.0,82.9,0.0,94000.0,40.640020,-74.094282
2405,2016,4880646,STATEN IS,Office,1975,100,36.0,10433999.8,4662157.1,92.8,0.0,94000.0,40.640020,-74.094282
2406,2017,4880646,STATEN IS,Office,1975,100,29.0,9899299.8,5098902.9,84.7,0.0,94000.0,40.640020,-74.094282
2407,2018,4880646,STATEN IS,Office,1975,100,32.0,8943200.3,5185757.1,84.7,0.0,94000.0,40.640020,-74.094282


In [92]:
final_data[final_data['borough'].isna()]

,Year,property_id,borough,primary_property_type_self,year_built,occupancy,energy_star_score,natural_gas_use_kbtu,electricity_use_grid_purchase,egrid_output_emissions_rate,green_power_offsite_kwh,largest_property_use_type_gross_floor_area_ft,latitude,longitude
569,2014,2715797,NaN,Multifamily Housing,1977,100,56.0,1515800.1,1289057.0,82.9,0.0,167828.0,NaN,NaN
570,2015,2715797,NaN,Multifamily Housing,1977,100,30.0,1214700.0,2517479.4,82.9,0.0,167828.0,NaN,NaN
571,2016,2715797,NaN,Multifamily Housing,1977,100,89.0,1763200.0,2551752.9,92.8,0.0,167828.0,NaN,NaN
572,2017,2715797,NaN,Multifamily Housing,1977,100,NaN,1711512.9,NaN,88.7,NaN,167828.0,NaN,NaN
573,2017,2715797,NaN,Multifamily Housing,1977,100,99.0,1711512.9,2458033.8,88.7,0.0,167828.0,NaN,NaN
574,2018,2715797,NaN,Multifamily Housing,1977,100,47.0,NaN,2442571.9,84.7,0.0,167828.0,NaN,NaN
784,2014,2787386,NaN,Multifamily Housing,1951,100,15.0,10376406.0,1434296.8,82.9,0.0,179273.0,NaN,NaN
785,2015,2787386,NaN,Multifamily Housing,1951,100,36.0,14739000.0,3173935.9,82.9,0.0,179273.0,NaN,NaN
786,2016,2787386,NaN,Multifamily Housing,1951,100,20.0,19866699.9,3368921.8,92.8,0.0,179273.0,NaN,NaN
2439,2019,4952804,NaN,Multifamily Housing,1951,100,99.0,3292599.9,1217500.5,79.5,0.0,121750.0,NaN,NaN


In [111]:
final_data.at[3644,'borough']='BROOKLYN'

In [71]:
final_data.iloc[1062,:]

Year                                                  2017
property_id                                        2809945
borough                                          MANHATTAN
primary_property_type_self                             NaN
year_built                                            1900
occupancy                                              100
energy_star_score                                     96.0
natural_gas_use_kbtu                             1068600.0
electricity_use_grid_purchase                    1605687.2
egrid_output_emissions_rate                           88.7
green_power_offsite_kwh                                0.0
largest_property_use_type_gross_floor_area_ft      92038.0
latitude                                         40.805842
longitude                                       -73.964767
Name: 1062, dtype: object

In [108]:
final_data[final_data['property_id']==5837739]

,Year,property_id,borough,primary_property_type_self,year_built,occupancy,energy_star_score,natural_gas_use_kbtu,electricity_use_grid_purchase,egrid_output_emissions_rate,green_power_offsite_kwh,largest_property_use_type_gross_floor_area_ft,latitude,longitude
3641,2016,5837739,BROOKLYN,College/University,1942,100,74.0,NaN,502806.7,92.8,0.0,58040.0,40.6398,-73.941625
3642,2017,5837739,NaN,College/University,1942,100,38.0,223699.3,930918.4,88.7,0.0,58040.0,NaN,NaN
3643,2017,5837739,NaN,College/University,1942,100,38.0,223699.3,930918.4,88.7,0.0,58040.0,NaN,NaN
3644,2018,5837739,NaN,College/University,1942,100,46.0,225102.5,851793.8,84.7,0.0,58040.0,NaN,NaN
3645,2019,5837739,BROOKLYN,College/University,1942,100,84.0,190800.0,726539.2,84.7,0.0,58040.0,40.6398,-73.941625


In [113]:
final_data.drop_duplicates()


,Year,property_id,borough,primary_property_type_self,year_built,occupancy,energy_star_score,natural_gas_use_kbtu,electricity_use_grid_purchase,egrid_output_emissions_rate,green_power_offsite_kwh,largest_property_use_type_gross_floor_area_ft,latitude,longitude
0,2014,1407848,MANHATTAN,Office,1963,100,53.0,8100654.8,211648520.8,82.9,0.0,1733828.0,40.754162,-73.976557
1,2015,1407848,MANHATTAN,Office,1963,85,55.0,7430345.4,173812489.7,82.9,0.0,1733828.0,40.754162,-73.976557
2,2016,1407848,MANHATTAN,Office,1963,85,72.0,7224061.1,137394745.4,92.8,0.0,1733828.0,40.754162,-73.976557
3,2017,1407848,MANHATTAN,Office,1963,85,59.0,7225638.6,159928080.3,88.7,0.0,1733828.0,40.754162,-73.976557
4,2018,1407848,MANHATTAN,Office,1963,85,32.0,7244369.0,161546077.8,84.7,0.0,1733828.0,40.754162,-73.976557
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3850,2018,6705804,BROOKLYN,Office,2000,100,80.0,22405.8,926002.5,84.7,0.0,33000.0,40.582060,-73.959960
3851,2019,6705804,BROOKLYN,Office,2000,100,75.0,13721.8,1151265.7,79.5,0.0,33000.0,40.582060,-73.959960
3852,2017,6705801,BROOKLYN,Office,2015,100,90.0,NaN,1621291.2,84.7,0.0,66869.0,40.580750,-73.965609
3855,2018,6705801,BROOKLYN,Office,2015,100,81.0,NaN,2091842.2,84.7,0.0,66869.0,40.580750,-73.965609


In [114]:
final_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3857 entries, 0 to 3856
Data columns (total 14 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   Year                                           3857 non-null   int64  
 1   property_id                                    3857 non-null   int64  
 2   borough                                        3857 non-null   object 
 3   primary_property_type_self                     3857 non-null   object 
 4   year_built                                     3857 non-null   int64  
 5   occupancy                                      3857 non-null   int64  
 6   energy_star_score                              3738 non-null   float64
 7   natural_gas_use_kbtu                           3496 non-null   float64
 8   electricity_use_grid_purchase                  3773 non-null   float64
 9   egrid_output_emissions_rate                    3851 

In [118]:
from google.colab import files

final_data.to_csv('Final_general_data.csv', encoding = 'utf-8-sig')
files.download('Final_general_data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>